In [3]:
# Install required libraries
!pip install transformers datasets torch

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, DatasetDict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [4]:
# Load multilingual BERT tokenizer and model
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Load and prepare dataset
# We'll use the XNLI dataset which contains text in multiple languages
datasets = load_dataset("xnli", "all_languages")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/6.77M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/3.39M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5010 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2490 [00:00<?, ? examples/s]

In [5]:

# Function to preprocess the data
#def preprocess_function(examples):
  #  return tokenizer(examples["premise"], examples["hypothesis"], truncation=True, padding="max_length", return_tensors="pt")

In [5]:
# Function to preprocess the data
def preprocess_function(examples):
    return tokenizer(
        [str(x) for x in examples["premise"]],  # Convert each premise to a string
        [str(x) for x in examples["hypothesis"]],  # Convert each hypothesis to a string
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

In [6]:
# Preprocess the datasets
tokenized_datasets = datasets.map(preprocess_function, batched=True)

# Prepare dataset splits
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))  # Use a subset for faster training
eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(1000))


Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/5010 [00:00<?, ? examples/s]

Map:   0%|          | 0/2490 [00:00<?, ? examples/s]

In [7]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

In [8]:
# Define metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [2]:
!pip install transformers datasets
from transformers import Trainer, TrainingArguments

In [9]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

Step,Training Loss
500,0.982500
1000,0.784100
1500,0.579800


{'eval_loss': 0.8127998113632202, 'eval_accuracy': 0.7, 'eval_f1': 0.7002264070749764, 'eval_precision': 0.7235766203726117, 'eval_recall': 0.7, 'eval_runtime': 29.4158, 'eval_samples_per_second': 33.995, 'eval_steps_per_second': 0.544, 'epoch': 3.0}


In [12]:
# Function to predict the class of a given text pair
def predict(premise, hypothesis):
    inputs = tokenizer(premise, hypothesis, return_tensors="pt", truncation=True, padding=True).to('cuda')
    outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return ["entailment", "neutral", "contradiction"][predicted_class]

In [13]:
# Example usage
premise = "The man is eating pasta."
hypothesis = "The man is consuming food."
result = predict(premise, hypothesis)
print(f"Premise: {premise}")
print(f"Hypothesis: {hypothesis}")
print(f"Prediction: {result}")

Premise: The man is eating pasta.
Hypothesis: The man is consuming food.
Prediction: entailment
